In [10]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
players = pd.read_csv('Players.csv')
player_data = pd.read_csv('player_data.csv')
season_stats = pd.read_csv('Seasons_Stats.csv')
season_stats = season_stats.drop(['blanl', 'blank2', 'Unnamed: 0'], axis=1)
season_stats['PPG'] = season_stats['PTS'] / season_stats['G']
season_stats['ASTPG'] = season_stats['AST'] / season_stats['G']
season_stats['STLPG'] = season_stats['STL'] / season_stats['G']
season_stats['BLKPG'] = season_stats['BLK'] / season_stats['G']
season_stats['PFPG'] = season_stats['PF'] / season_stats['G']
season_stats['TRPG'] = season_stats['TRB'] / season_stats['G']
season_stats['ORPG'] = season_stats['ORB'] / season_stats['G']
season_stats['DRPG'] = season_stats['DRB'] / season_stats['G']
season_stats = season_stats.dropna()

In [51]:
players.head()

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [52]:
player_data.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


In [53]:
season_stats.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG,ASTPG,STLPG,BLKPG,PFPG,TRPG,ORPG,DRPG
5730,1980.0,Tiny Archibald*,PG,31.0,BOS,80.0,80.0,2864.0,15.3,0.574,0.023,0.548,2.3,5.3,3.8,30.2,1.7,0.2,19.7,17.0,5.9,2.9,8.9,0.148,1.1,-1.1,0.0,1.5,383.0,794.0,0.482,4.0,18.0,0.222,379.0,776.0,0.488,0.485,361.0,435.0,0.830,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0,14.137500,8.387500,1.325000,0.125000,2.725000,2.462500,0.737500,1.725000
5748,1980.0,Larry Bird*,PF,23.0,BOS,82.0,82.0,2955.0,20.5,0.538,0.098,0.246,8.3,23.6,16.0,18.5,2.3,1.0,14.0,25.3,5.6,5.6,11.2,0.182,2.7,2.6,5.3,5.4,693.0,1463.0,0.474,58.0,143.0,0.406,635.0,1320.0,0.481,0.494,301.0,360.0,0.836,216.0,636.0,852.0,370.0,143.0,53.0,263.0,279.0,1745.0,21.280488,4.512195,1.743902,0.646341,3.402439,10.390244,2.634146,7.756098
5781,1980.0,M.L. Carr,SF,29.0,BOS,82.0,7.0,1994.0,14.6,0.526,0.054,0.316,6.0,12.3,9.2,10.7,2.8,1.0,14.1,20.2,1.9,3.2,5.2,0.125,-0.2,1.5,1.3,1.7,362.0,763.0,0.474,12.0,41.0,0.293,350.0,722.0,0.485,0.482,178.0,241.0,0.739,106.0,224.0,330.0,156.0,120.0,36.0,143.0,214.0,914.0,11.146341,1.902439,1.463415,0.439024,2.609756,4.024390,1.292683,2.731707
5786,1980.0,Don Chaney,SG,33.0,BOS,60.0,0.0,523.0,7.8,0.402,0.032,0.222,6.7,8.8,7.8,9.3,2.8,1.2,13.7,18.3,-0.4,0.8,0.4,0.033,-4.0,0.9,-3.1,-0.1,67.0,189.0,0.354,1.0,6.0,0.167,66.0,183.0,0.361,0.357,32.0,42.0,0.762,31.0,42.0,73.0,38.0,31.0,11.0,33.0,80.0,167.0,2.783333,0.633333,0.516667,0.183333,1.333333,1.216667,0.516667,0.700000
5802,1980.0,Dave Cowens*,C,31.0,BOS,66.0,55.0,2159.0,14.8,0.477,0.013,0.131,6.6,20.7,13.8,13.4,1.5,1.6,9.9,20.1,1.9,3.6,5.4,0.121,-0.7,2.0,1.4,1.8,422.0,932.0,0.453,1.0,12.0,0.083,421.0,920.0,0.458,0.453,95.0,122.0,0.779,126.0,408.0,534.0,206.0,69.0,61.0,108.0,216.0,940.0,14.242424,3.121212,1.045455,0.924242,3.272727,8.090909,1.909091,6.181818


In [54]:
season_stats[season_stats['Player'] == 'LeBron James']


,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG,ASTPG,STLPG,BLKPG,PFPG,TRPG,ORPG,DRPG
16746,2004.0,LeBron James,SG,19.0,CLE,79.0,79.0,3122.0,18.3,0.488,0.145,0.308,3.5,11.8,7.6,27.8,2.2,1.3,13.9,28.2,2.4,2.6,5.1,0.078,2.2,-0.2,1.9,3.1,622.0,1492.0,0.417,63.0,217.0,0.290,559.0,1275.0,0.438,0.438,347.0,460.0,0.754,99.0,333.0,432.0,465.0,130.0,58.0,273.0,149.0,1654.0,20.936709,5.886076,1.645570,0.734177,1.886076,5.468354,1.253165,4.215190
17344,2005.0,LeBron James,SF,20.0,CLE,80.0,80.0,3388.0,25.7,0.554,0.183,0.378,3.8,17.0,10.2,32.9,2.8,1.1,11.8,29.7,9.7,4.6,14.3,0.203,6.9,1.5,8.3,8.8,795.0,1684.0,0.472,108.0,308.0,0.351,687.0,1376.0,0.499,0.504,477.0,636.0,0.750,111.0,477.0,588.0,577.0,177.0,52.0,262.0,146.0,2175.0,27.187500,7.212500,2.212500,0.650000,1.825000,7.350000,1.387500,5.962500
17918,2006.0,LeBron James,SF,21.0,CLE,79.0,79.0,3361.0,28.1,0.568,0.208,0.447,2.6,17.1,9.8,32.8,2.0,1.5,10.7,33.6,12.0,4.3,16.3,0.232,7.9,1.4,9.3,9.5,875.0,1823.0,0.480,127.0,379.0,0.335,748.0,1444.0,0.518,0.515,601.0,814.0,0.738,75.0,481.0,556.0,521.0,123.0,66.0,260.0,181.0,2478.0,31.367089,6.594937,1.556962,0.835443,2.291139,7.037975,0.949367,6.088608
18460,2007.0,LeBron James,SF,22.0,CLE,78.0,78.0,3190.0,24.5,0.552,0.191,0.432,3.0,16.6,9.6,29.1,2.1,1.3,11.5,31.0,8.0,5.7,13.7,0.206,5.4,2.0,7.4,7.6,772.0,1621.0,0.476,99.0,310.0,0.319,673.0,1311.0,0.513,0.507,489.0,701.0,0.698,83.0,443.0,526.0,470.0,125.0,55.0,250.0,171.0,2132.0,27.333333,6.025641,1.602564,0.705128,2.192308,6.743590,1.064103,5.679487
19017,2008.0,LeBron James,SF,23.0,CLE,75.0,74.0,3027.0,29.1,0.568,0.219,0.470,4.9,17.8,11.1,37.3,2.4,2.1,11.4,33.5,10.7,4.6,15.2,0.242,9.0,2.3,11.2,10.1,794.0,1642.0,0.484,113.0,359.0,0.315,681.0,1283.0,0.531,0.518,549.0,771.0,0.712,133.0,459.0,592.0,539.0,138.0,81.0,255.0,165.0,2250.0,30.000000,7.186667,1.840000,1.080000,2.200000,7.893333,1.773333,6.120000
19611,2009.0,LeBron James,SF,24.0,CLE,81.0,81.0,3054.0,31.7,0.591,0.238,0.472,4.3,19.0,11.9,38.0,2.4,2.4,11.0,33.8,13.7,6.5,20.3,0.318,9.4,3.6,13.0,11.6,789.0,1613.0,0.489,132.0,384.0,0.344,657.0,1229.0,0.535,0.530,594.0,762.0,0.780,106.0,507.0,613.0,587.0,137.0,93.0,241.0,139.0,2304.0,28.444444,7.246914,1.691358,1.148148,1.716049,7.567901,1.308642,6.259259
20207,2010.0,LeBron James,SF,25.0,CLE,76.0,76.0,2966.0,31.1,0.604,0.253,0.506,3.0,18.5,11.1,41.8,2.2,2.0,12.3,33.5,13.3,5.2,18.5,0.299,9.7,2.8,12.5,10.9,768.0,1528.0,0.503,129.0,387.0,0.333,639.0,1141.0,0.560,0.545,593.0,773.0,0.767,71.0,483.0,554.0,651.0,125.0,77.0,261.0,119.0,2258.0,29.710526,8.565789,1.644737,1.013158,1.565789,7.289474,0.934211,6.355263
20812,2011.0,LeBron James,SF,26.0,MIA,79.0,79.0,3063.0,27.3,0.594,0.188,0.446,3.3,18.7,11.4,34.9,2.1,1.3,13.8,31.5,10.3,5.3,15.6,0.244,6.5,2.1,8.6,8.2,758.0,1485.0,0.510,92.0,279.0,0.330,666.0,1206.0,0.552,0.541,503.0,663.0,0.759,80.0,510.0,590.0,554.0,124.0,50.0,284.0,163.0,2111.0,26.721519,7.012658,1.569620,0.632911,2.063291,7.468354,1.012658,6.455696
21395,2012.0,LeBron James,SF,27.0,MIA,62.0,62.0,2326.0,30.7,0.605,0.127,0.429,5.0,19.7,12.6,33.6,2.6,1.7,13.3,32.0,10.0,4.5,14.5,0.298,8.3,2.7,11.0,7.6,621.0,1169.0,0.531,54.0,149.0,0.362,567.0,1020.0,0.556,0.554,387.0,502.0,0.771,94.0,398.0,492.0,387.0,115.0,50.0,213.0,96.0,1683.0,27.145161,6.241935,1.854839,0.806452,1.548387,7.935484,1.516129,6.419355
21933,2013.0,LeBron James,PF,28.0,MIA,76.0,76.0,2877.0,31.6,0.640,0.188,0.395,4.4,20.8,13.1,36.4,2.4,1.9,12.4,30.2,14.6,4.7,19.3,0.322,9.2,2.4,11.6,9.8,765.0,1354.0,0.565,103.0,254.0,0.406,662.0,1100.0,0.602,0.603,403.0,535.0,0.753,97.0,513.0,610.0,551.0,129.0,67.0,226.0,110.0,2036.0,26.789474,7.250000,1.697368,0.881579,1.447368,8.026316,1.276316,6.750000
